In [4]:
!pip install dwave-ocean-sdk yfinance pandas scikit-learn keras numpy plotly yfinance -q

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from dwave.system import DWaveSampler, EmbeddingComposite
import plotly.graph_objects as go
from datetime import timedelta

# Loaded Secrets
from kaggle_secrets import UserSecretsClient

# Set up D-Wave API token
user_secrets = UserSecretsClient()
API_TOKEN = user_secrets.get_secret("dwave-api")

# Retrieve BTC-USD stock data
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="1y")
    return data[['Close']]

# Preprocess the data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Create dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Define LSTM model
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(input_shape, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Optimize hyperparameters using D-Wave
def optimize_hyperparameters():
    sampler = EmbeddingComposite(DWaveSampler(token=API_TOKEN))
    
    # Define a simple QUBO problem for demonstration (real optimization would be more complex)
    Q = {(0, 0): 1, (1, 1): 1, (0, 1): -1}
    
    # Solve the QUBO problem
    response = sampler.sample_qubo(Q, num_reads=100)
    return response.first.sample

# Main function
def main():
    data = get_stock_data('BTC-USD')
    last_3_months = data.index[-3*30:]  # Last 3 months data
    scaled_data, scaler = preprocess_data(data)
    
    time_step = 60
    X, Y = create_dataset(scaled_data, time_step)
    
    X = X.reshape(X.shape[0], X.shape[1], 1)
    
    model = create_model(time_step)
    
    # Optimize hyperparameters (if needed)
    optimized_params = optimize_hyperparameters()
    print("Optimized Parameters:", optimized_params)
    
    model.fit(X, Y, epochs=10, batch_size=32, verbose=2)
    
    # Predict the next 7 days
    predictions = []
    for _ in range(7):
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, time_step, 1)
        pred_price = model.predict(last_60_days)
        predictions.append(pred_price[0, 0])
        scaled_data = np.append(scaled_data, pred_price)
    
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    print("Predicted Prices for the next 7 days:", predictions)

    # Create dates for the next 7 days
    last_date = data.index[-1]
    prediction_dates = [last_date + timedelta(days=i) for i in range(1, 8)]
    
    # Plot the results
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=data.index[-90:], y=data['Close'][-90:], mode='lines', name='Historical Prices'))
    fig.add_trace(go.Scatter(x=prediction_dates, y=predictions.flatten(), mode='lines', name='Predicted Prices'))
    
    fig.update_layout(
        title='BTC-USD Stock Price Prediction',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis=dict(
            range=[last_3_months[0], prediction_dates[-1]]
        )
    )
    
    fig.show()

if __name__ == "__main__":
    main()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Optimized Parameters: {0: 0, 1: 0}
Epoch 1/10
10/10 - 3s - 327ms/step - loss: 0.1180
Epoch 2/10
10/10 - 0s - 39ms/step - loss: 0.0167
Epoch 3/10
10/10 - 0s - 40ms/step - loss: 0.0100
Epoch 4/10
10/10 - 0s - 42ms/step - loss: 0.0060
Epoch 5/10
10/10 - 0s - 40ms/step - loss: 0.0047
Epoch 6/10
10/10 - 0s - 41ms/step - loss: 0.0039
Epoch 7/10
10/10 - 0s - 40ms/step - loss: 0.0037
Epoch 8/10
10/10 - 0s - 41ms/step - loss: 0.0036
Epoch 9/10
10/10 - 0s - 39ms/step - loss: 0.0034
Epoch 10/10
10/10 - 0s - 44ms/step - loss: 0.0034
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted Prices for the next 7 days: [[66954.73 ]
 [66718.48 ]
 [66604.586]
 [66575.63 ]
 [66605.9  ]
 [66677.78 ]
 [66778.92 ]]


# Claude Iteration .1

This code implements a stock price prediction model for Apple (BTC-USD) stock using a combination of technical analysis indicators and machine learning. Here's a summary of its strategy and components:

1. Data Retrieval:
   - Fetches one year of historical stock data (High, Low, Close prices) for BTC-USD using the yfinance library.

2. Technical Analysis:
   - Calculates the Central Pivot Range (CPR), specifically the R1 level, which is a common pivot point indicator.
   - Implements a Range Filter indicator to smooth out price movements.

3. Data Preprocessing:
   - Normalizes the data using MinMaxScaler to scale all features to a range between 0 and 1.
   - Creates a dataset suitable for LSTM training, using a 60-day lookback period.

4. Machine Learning Model:
   - Utilizes a Long Short-Term Memory (LSTM) neural network, which is well-suited for time series prediction.
   - The model architecture consists of two LSTM layers followed by dense layers.
   - Predicts the next day's High, Low, and Close prices based on the previous 60 days of data.

5. Quantum Computing Integration:
   - Uses D-Wave's quantum annealer for hyperparameter optimization, although in this example it's a simplified demonstration.

6. Prediction:
   - Uses the trained model to predict stock prices for the next 7 days.

7. Visualization:
   - Plots the historical filtered prices and the predicted prices using Plotly.

Strategy:
The overall strategy combines traditional technical analysis with modern machine learning techniques:
1. It uses the CPR and Range Filter to capture important price levels and reduce noise in the data.
2. The LSTM model learns patterns from this processed data, potentially capturing both short-term and long-term dependencies in the price movements.
3. By predicting High, Low, and Close prices, it aims to provide a more comprehensive view of future price movements.
4. The inclusion of quantum computing for optimization, while simplified here, suggests an attempt to leverage advanced computational methods for better model performance.

It's important to note that while this approach is sophisticated, predicting stock prices accurately is extremely challenging. This model should be considered as a starting point for further refinement and not as a definitive prediction tool for real-world trading without extensive validation and improvement.

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential 
from keras.layers import LSTM, Dense
from dwave.system import DWaveSampler, EmbeddingComposite
import plotly.graph_objects as go
from datetime import timedelta

# Loaded Secrets
from kaggle_secrets import UserSecretsClient

# Set up D-Wave API token
user_secrets = UserSecretsClient()
API_TOKEN = user_secrets.get_secret("dwave-api")

# Retrieve BTC-USD stock data including High, Low, Close
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="1y")
    return data[['High', 'Low', 'Close']]

# Calculate Central Pivot Range (CPR)
def calculate_cpr(data):
    high = data['High']
    low = data['Low']
    close = data['Close']
    pivot = (high + low + close) / 3
    r1 = 2 * pivot - low
    return r1.shift(-1)  # Tomorrow's R1

# Implement Range Filter Indicator
def range_filter(data, r1, sampling_period=192, range_multiplier=9):
    filtered_data = data.copy()
    for i in range(len(data) - sampling_period):
        if data['Close'][i] > r1[i] * range_multiplier:
            filtered_data['Close'][i] = r1[i] * range_multiplier
    return filtered_data

# Preprocess the data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Create dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), :]
        X.append(a)
        Y.append(data[i + time_step, :])  # Predict all three values
    return np.array(X), np.array(Y)

# Define LSTM model
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(input_shape, 3)))  # 3 for High, Low, Close
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(3))  # Output 3 values
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Optimize hyperparameters using D-Wave
def optimize_hyperparameters():
    sampler = EmbeddingComposite(DWaveSampler(token=API_TOKEN))
    
    # Define a simple QUBO problem for demonstration (real optimization would be more complex)
    Q = {(0, 0): 1, (1, 1): 1, (0, 1): -1}
    
    # Solve the QUBO problem
    response = sampler.sample_qubo(Q, num_reads=100)
    return response.first.sample

# Main function
def main():
    data = get_stock_data('BTC-USD')
    
    # Calculate CPR and get Tomorrow's R1
    r1 = calculate_cpr(data)
    
    # Apply Range Filter indicator
    filtered_data = range_filter(data, r1)
    
    # Preprocess the data
    scaled_data, scaler = preprocess_data(filtered_data[['High', 'Low', 'Close']])
    
    time_step = 60
    X, Y = create_dataset(scaled_data, time_step)
    
    model = create_model(time_step)
    
    # Optimize hyperparameters (if needed)
    optimized_params = optimize_hyperparameters()
    print("Optimized Parameters:", optimized_params)
    
    model.fit(X, Y, epochs=10, batch_size=32, verbose=2)
    
    # Predict the next 7 days
    predictions = []
    last_60_days = scaled_data[-time_step:]
    for _ in range(7):
        pred_price = model.predict(last_60_days.reshape(1, time_step, 3))
        predictions.append(pred_price[0])
        last_60_days = np.vstack((last_60_days[1:], pred_price))
    
    predictions = scaler.inverse_transform(np.array(predictions))
    print("Predicted Prices for the next 7 days:", predictions)

    # Create dates for the next 7 days
    last_date = data.index[-1]
    prediction_dates = [last_date + timedelta(days=i) for i in range(1, 8)]
    
    # Plot the results
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=filtered_data.index[-90:], y=filtered_data['Close'][-90:], mode='lines', name='Filtered Prices'))
    fig.add_trace(go.Scatter(x=prediction_dates, y=predictions[:, 2], mode='lines', name='Predicted Close Prices'))
    
    fig.update_layout(
        title='BTC-USD Stock Price Prediction with Enhanced Model',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis=dict(
            range=[filtered_data.index[-90], prediction_dates[-1]]
        )
    )
    
    fig.show()

if __name__ == "__main__":
    main()

/tmp/ipykernel_33/3418315828.py:37: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Optimized Parameters: {0: 0, 1: 0}
Epoch 1/10
10/10 - 3s - 323ms/step - loss: 0.1169
Epoch 2/10
10/10 - 1s - 64ms/step - loss: 0.0148
Epoch 3/10
10/10 - 0s - 38ms/step - loss: 0.0083
Epoch 4/10
10/10 - 0s - 40ms/step - loss: 0.0054
Epoch 5/10
10/10 - 0s - 39ms/step - loss: 0.0038
Epoch 6/10
10/10 - 0s - 38ms/step - loss: 0.0036
Epoch 7/10
10/10 - 0s - 39ms/step - loss: 0.0034
Epoch 8/10
10/10 - 0s - 38ms/step - loss: 0.0034
Epoch 9/10
10/10 - 0s - 38ms/step - loss: 0.0032
Epoch 10/10
10/10 - 0s - 41ms/step - loss: 0.0033
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Prices for the next 7 days: [[68909.58  65630.96  67685.83 ]
 [68778.39  65479.68  67532.31 ]
 [68743.72  65439.305 67489.51 ]
 [68777.2   65472.777 67523.875]
 [68855.71  65553.52  67609.72 ]
 [6